In [1]:
%reset

In [2]:
import pickle
import os
import torch
import warnings
import numpy as np

In [3]:
server = "bohr"

# Number of files
num_files = 416
# Base filename components
base_name_data_BG = "train_data_BG_" + server
base_name_data_DM = "train_data_DM_" + server
ext_data = ".pkl"

base_name_thetas_BG = "train_thetas_BG_" + server
base_name_thetas_DM = "train_thetas_DM_" + server
ext_thetas = ".pt"

In [4]:
warnings.filterwarnings("ignore", category=FutureWarning)

# Prepare list to hold all dictionaries
data_list = []
thetas_list = []

for i in range(num_files):
    # Data files
    # For background signal
    filename_data_BG = f"{base_name_data_BG}_{i}{ext_data}"
    if not os.path.exists(filename_data_BG):
        raise FileNotFoundError(f"Expected file not found: {filename_data_BG}")
    with open(filename_data_BG, "rb") as f:
        data_BG = pickle.load(f)  # Unpickle the dictionary
    # For DM signal
    filename_data_DM = f"{base_name_data_DM}_{i}{ext_data}"
    if not os.path.exists(filename_data_DM):
        raise FileNotFoundError(f"Expected file not found: {filename_data_DM}")
    with open(filename_data_DM, "rb") as f:
        data_DM = pickle.load(f)  # Unpickle the dictionary

    combined_angles = np.concatenate((data_BG['angles'], data_DM['angles']), axis=0) # vetcical stack
    combined_energies  = np.concatenate((data_BG['energies'], data_DM['energies']), axis=0) # vertical stack
    data_1_sim = {
    'angles': combined_angles,
    'energies': combined_energies
    }

    data_list.append(data_1_sim)



    # Theta files
    # For background signal
    filename_thetas_BG = f"{base_name_thetas_BG}_{i}{ext_thetas}"
    if not os.path.exists(filename_thetas_BG):
        raise FileNotFoundError(f"Missing file: {filename_thetas_BG}")
    theta_BG = torch.load(filename_thetas_BG) 
    # Ensure it’s the expected shape
    if theta_BG.ndim != 1 or theta_BG.numel() != 1:
        raise ValueError(f"Unexpected shape in {filename_thetas_BG}: {theta_BG.shape}")
    # For DM signal
    filename_thetas_DM = f"{base_name_thetas_DM}_{i}{ext_thetas}"
    if not os.path.exists(filename_thetas_DM):
        raise FileNotFoundError(f"Missing file: {filename_thetas_DM}")
    theta_DM = torch.load(filename_thetas_DM) 
    # Ensure it’s the expected shape
    if theta_DM.ndim != 1 or theta_DM.numel() != 2:
        raise ValueError(f"Unexpected shape in {filename_thetas_DM}: {theta_DM.shape}")
    
    thetas_1_sim = torch.cat((theta_BG, theta_DM), dim=0)
    
    thetas_list.append(thetas_1_sim)

# Stack into a single (260, 2) tensor
thetas_tensor = torch.stack(thetas_list, dim=0)

In [5]:
print(f"len(data_list): {len(data_list)}")
print(f"thetas_tensor.shape: {thetas_tensor.shape}")

len(data_list): 416
thetas_tensor.shape: torch.Size([416, 3])


In [6]:
# Save data
output_file_data = "combined_train_data_" + server + ".pkl"
with open(output_file_data, "wb") as f:
    pickle.dump(data_list, f)  # Serialize the list of dicts 

print(f"Loaded {len(data_list)} files and saved combined data to '{output_file_data}'.")

# Save thetas
output_file_thetas = "combined_train_thetas_" + server + ".pt"
torch.save(thetas_tensor, output_file_thetas)

print(f"Saved combined thetas tensor of shape {thetas_tensor.shape} to '{output_file_thetas}'.")

Loaded 416 files and saved combined data to 'combined_train_data_bohr.pkl'.
Saved combined thetas tensor of shape torch.Size([416, 3]) to 'combined_train_thetas_bohr.pt'.
